# **0. Imports**

In [2]:
import pandas as pd
import numpy as np

## **0.1 Load Dataset**

In [3]:
data = pd.read_csv('../data/data_trab2_econometria.csv')
data.head()

,Data,inflacao,cambio,juros
0,2003,"1,98","3,44","26,38"
1,2003,"2,19","3,59","24,31"
2,2003,"1,14","3,45","23,58"
3,2003,"1,14","3,12","24,90"
4,2003,"0,85","2,95","26,38"


# **1.0 Data Description**

## **1.1 Rename columns**

In [23]:
df1 = data.copy()

In [24]:
cols_new = ['date','cpi', 'exchange_rate','interest_rate']
df1.columns = cols_new

## **1.2 Data Dimensions**

In [25]:
print('Number of rows:', format(df1.shape[0]))
print('Number of columns:', format(df1.shape[1]))

Number of rows: 223
Number of columns: 4


## **1.3 Data types**

In [26]:
df1.dtypes

date              int64
cpi              object
exchange_rate    object
interest_rate    object
dtype: object

## **1.4 Check NA**

In [27]:
df1.isna().sum()

date             0
cpi              0
exchange_rate    0
interest_rate    0
dtype: int64

## **1.5 Change Types**

In [28]:
df1.head()

,date,cpi,exchange_rate,interest_rate
0,2003,"1,98","3,44","26,38"
1,2003,"2,19","3,59","24,31"
2,2003,"1,14","3,45","23,58"
3,2003,"1,14","3,12","24,90"
4,2003,"0,85","2,95","26,38"


In [34]:
df1['date'] = pd.to_datetime(df1['date'].astype(str))
#change comma to point decimal

cols = ['cpi', 'exchange_rate', 'interest_rate']

for col in cols:
    df1[col] = df1[col].apply(lambda x: x.replace(',', '.'))



In [35]:
df1.head()

,date,cpi,exchange_rate,interest_rate
0,2003-01-01,1.98,3.44,26.38
1,2003-01-01,2.19,3.59,24.31
2,2003-01-01,1.14,3.45,23.58
3,2003-01-01,1.14,3.12,24.90
4,2003-01-01,0.85,2.95,26.38


## **1.6 Descriptive Statistical**